## Articles are in MongoDB; THIS is what we want!!

In [1]:
import sys

In [2]:
sys.path.append('/home/immersinn/gits/weblog_analytics/src')

In [3]:
import utils

In [27]:
articles = utils.retrieve_article_htmls(coll_name='PhysOrg', limit=0)

In [28]:
articles.head()

,LastUpdate,_id,html,url
0,2016-11-16 00:23:26,582ba6fe50c3c9212d7ad994,b'<!DOCTYPE html>\n<html>\n <head>\n\t<meta c...,http://phys.org/news/2016-11-pre-mammalian-anc...
1,2016-11-16 00:23:28,582ba70050c3c9212d7ad995,b'<!DOCTYPE html>\n<html>\n <head>\n\t<meta c...,http://phys.org/news/2016-10-ancient-burials-b...
2,2016-11-16 00:23:29,582ba70150c3c9212d7ad996,b'<!DOCTYPE html>\n<html>\n <head>\n\t<meta c...,http://phys.org/news/2016-10-fossilized-dinosa...
3,2016-11-16 00:23:30,582ba70250c3c9212d7ad997,b'<!DOCTYPE html>\n<html>\n <head>\n\t<meta c...,http://phys.org/news/2016-11-mathematics-coffe...
4,2016-11-16 00:23:31,582ba70350c3c9212d7ad998,b'<!DOCTYPE html>\n<html>\n <head>\n\t<meta c...,http://phys.org/news/2016-11-image-solar-cells...


## Extract Relevant Content

In [6]:
from bs4 import BeautifulSoup as bs

In [7]:
import article_handles

In [8]:
proc_articles = [article_handles.articleObjectPhysOrg(url, bs(html, 'html.parser')) \
                 for url, html in zip(list(articles.url), list(articles.html))]

In [9]:
for pa in proc_articles:
    pa.processArticle()

In [10]:
pa = proc_articles[5]

In [11]:
pa.meta

{'categories': ('Home', 'Other Sciences', 'Archaeology & Fossils'),
 'source': 'PhysOrg',
 'title': None,
 'url': 'http://phys.org/news/2016-11-biggest-dinosaur-tree-emerged-million.html'}

In [12]:
len(pa.contents)

2067

In [13]:
pa.contents[:200]

'(Phys.org)—A team of researchers from the U.K. and the U.S. has mapped the biggest dinosaur tree yet, and in so doing, has found that the creatures may have evolved 20 million years earlier than most '

In [14]:
pa.url

'http://phys.org/news/2016-11-biggest-dinosaur-tree-emerged-million.html'

## spaCy, textacy Stuffs

In [15]:
import spacy
import textacy

In [16]:
textacy_preprocessor = lambda text: textacy.preprocess.preprocess_text(text,
                                                                       no_contractions=True,
                                                                       no_numbers=True,
                                                                       no_emails=True,
                                                                       no_currency_symbols=True,
                                                                       lowercase=True)

In [17]:
nlp = spacy.load("en", add_vectors=False)

In [18]:
nlp.pipeline = [nlp.tagger, nlp.parser]

In [19]:
texts = [textacy_preprocessor(pa.contents) for pa in proc_articles]

In [20]:
metas = [{'title':pa.title, 'url':pa.url} for pa in proc_articles]

In [21]:
corpus = textacy.Corpus(lang=nlp)

In [22]:
corpus.add_texts(texts=texts, metadatas=metas, n_threads=8, batch_size=50)

In [23]:
corpus.n_docs

20

In [24]:
corpus.n_sents

593

In [25]:
corpus.save(path='/home/immersinn/gits/weblog_analytics/data/processed/',
            name='CORPUS_physorg_articles_test', compression='gzip')

In [26]:
corpus = textacy.Corpus.load(path='/home/immersinn/gits/weblog_analytics/data/processed/',
                             name='CORPUS_physorg_articles_test',
                             compression='gzip')